In [ ]:
!git clone https://github.com/hahnicity/ventmode
!python3 -m pip install ./ventmode
!python -m pip install git+https://github.com/lukauskas/dtwco.git#egg=dtwco
%pip install tslearn visualkeras

In [ ]:
!git clone https://github.com/Villiamfj/ventTools
%pip install -e ventTools

In [ ]:
%pip install -q -U keras-tuner
import keras_tuner as kt

In [65]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import numpy as np
import ventTools
from ventTools.data import convert_to_fixed_length
from ast import literal_eval
import ventmode

# Encoders

In [66]:
def get_lstm_encoder(lstm_units, dropout, final_size):
    model = keras.Sequential()
    model.add(layers.LayerNormalization(epsilon=1e-6, axis=-1))
    
    for dim in lstm_units:
      model.add(layers.LSTM(dim, return_sequences= True))
      model.add(layers.Dropout(dropout))
    model.add(layers.LSTM(final_size))
    return model

In [67]:
def get_conv_encoder(conv_filters, kernel, dropout, hidden_activation):
  model = keras.Sequential()
  model.add(layers.LayerNormalization(epsilon=1e-6, axis=-1))

  first_layer = conv_filters.pop(0)
  model.add(layers.Conv1D(first_layer, kernel, activation = hidden_activation, padding = "same"))
    
  for filter in conv_filters:
    model.add(layers.Dropout(dropout))
    model.add(layers.Conv1D(filter, kernel, activation = hidden_activation, padding = "same"))
    
  model.add(layers.GlobalAveragePooling1D())
  return model

In [68]:
def get_feed_forward_encoder(units, dropout, final_size, hidden_activation):
  model = keras.Sequential()
  model.add(layers.LayerNormalization(epsilon=1e-6, axis=-1))

  for unit in units:
    model.add(layers.Dense(unit, activation = hidden_activation))
    model.add(layers.Dropout(dropout))
  model.add(layers.Dense(final_size, activation = "softmax"))
  model.add(layers.GlobalAveragePooling1D())
  #model.add(layers.Dense(final_size, activation = "softmax"))

  return model

# Transformer

In [69]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

def apply_positional_encoding(x, position, d_model):
  return x + positional_encoding(position, d_model)

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x) # Is this necesarry?
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


In [70]:
def get_tunable_transformer(hp):
    input_shape = (202,2)
    n_classes = 5

    # 2 tune
    head_size = hp.Int('head_size', min_value = 32, max_value = 256, step = 32)
    num_heads = hp.Int("num_heads", min_value = 2, max_value = 8, step = 2 )
    ff_dim = hp.Int('ff_dim', min_value = 2, max_value=8, step = 2)

    num_transformer_blocks = hp.Int("num_transformer_block", min_value = 1, max_value=4, step=1)
    
    mlp_dropout = hp.Float("dropout", min_value = 0.0, max_value = 0.5)
    mlp_units = [
                hp.Choice("mlp_1", values = [128, 0]),
                hp.Choice("mlp_2", values = [64, 0]),
                hp.Choice("mlp_3", values = [32, 0]),
    ]    
    dropout=hp.Float('dropout', min_value = 0., max_value=0.5, step=0.1)
    
    # building model
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = layers.LayerNormalization(epsilon=1e-6, axis=-2)(x)
    x = apply_positional_encoding(x, input_shape[-2], input_shape[-1])
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    model = keras.Model(inputs, outputs)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    optimizer = keras.optimizers.Adam(keras.optimizers.schedules.CosineDecay(initial_learning_rate=hp_learning_rate, decay_steps=1000, alpha=1e-2))
    model.compile(
      loss=keras.losses.SparseCategoricalCrossentropy(),
      optimizer=optimizer,
      metrics=["sparse_categorical_accuracy"],
    )
    
    return model

# Classifier

In [71]:
# Classifier function
def get_mlp_classifier(n_classes, mlp_dropout, dimensions):
  model = keras.Sequential()

  for dim in dimensions:
        model.add(layers.Dense(dim, activation="relu"))
        model.add(layers.Dropout(mlp_dropout))

  model.add(layers.Dense(n_classes, activation="softmax"))
  return model

# Tuning builders

In [72]:
from itertools import chain, combinations

def powerset(iterable):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

def get_powerset(option_list):
  power = list(powerset(option_list))
  resulting_set = []
  for item in power:
    resulting_set.append(list(item))
  return resulting_set

In [73]:
def get_tunable_classifier(hp):
  n_classes = 5
  mlp_dropout = hp.Float("dropout", min_value = 0.0, max_value = 0.8)

  mlp_units = [
               hp.Choice("mlp_1", values = [128, 0]),
               hp.Choice("mlp_2", values = [64, 0]),
               hp.Choice("mlp_3", values = [32, 0]),
  ]

  return get_mlp_classifier(n_classes, mlp_dropout, mlp_units)

In [74]:
def get_model_built(model, learning_rate, alpha):
  model.build(input_shape  = (None,2,202))
  optimizer=keras.optimizers.Adam(learning_rate=keras.optimizers.schedules.CosineDecay(initial_learning_rate=learning_rate, decay_steps=1000, alpha=alpha))
  model.compile(optimizer = optimizer,
                loss=keras.losses.SparseCategoricalCrossentropy(),
                metrics=['sparse_categorical_accuracy'])
  return model


def get_tunable_lstm_builder(get_classifier_func):
  def get_tunable_lstm(hp):
    lstm_units = [
                  hp.Choice("lstm_1", values = [128, 0]),
                  hp.Choice("lstm_2", values = [64, 0]),
                  hp.Choice("lstm_3", values = [32, 0]),
    ]
    lstm_units = filter(lambda x: x != 0, lstm_units)

    dropout = hp.Float("dropout", min_value = 0.0, max_value = 0.5)
    final_size = hp.Int("encoder output size", min_value = 3, max_value = 100)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_alpha = hp.Choice('alpha', values=[1e-1, 1e-2])

    model = keras.Sequential([
                              get_lstm_encoder(lstm_units, dropout, final_size),
                              get_classifier_func(hp)   
    ])
    model = get_model_built(model, hp_learning_rate, hp_alpha)
    return model
  return get_tunable_lstm

def get_tunable_conv_builder_mod(get_classifier_func):
  def get_tunable_conv(hp):
    filter_size = hp.Int("filter", min_value = 32, max_value = 128)
    filter_count = hp.Int("filter_count", min_value = 1, max_value = 3)
    conv_filter = [filter_size for _ in range(filter_count)]

    kernel = hp.Int("kernel", min_value = 1, max_value = 3)
    dropout = hp.Float("dropout", min_value = 0.0, max_value = 0.5)

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_alpha = hp.Choice('alpha', values=[1e-1, 1e-2])

    hidden_activation = hp.Choice("hidden_activation", values = ["relu", "sigmoid", "gelu"])

    model = keras.Sequential([
                              get_conv_encoder(conv_filter, kernel, dropout,hidden_activation),
                              get_classifier_func(hp)   
    ])
    model = get_model_built(model, hp_learning_rate, hp_alpha)
    return model
  return get_tunable_conv


def get_tunable_ff_builder(get_classifier_func):
  def get_tunable_ff(hp):
    ff_units = [
                  hp.Choice("ff_1", values = [128, 0]),
                  hp.Choice("ff_2", values = [64, 0]),
                  hp.Choice("ff_3", values = [32, 0]),              
    ]
    ff_units = filter(lambda x : x != 0, ff_units)

    activation = hp.Choice("hidden activation", values = ["relu","sigmoid","gelu"])
    dropout = hp.Float("dropout", min_value = 0.0, max_value = 0.5)
    
    final_size = hp.Int("encoder output size", min_value = 3, max_value = 100)


    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    hp_alpha = hp.Choice('alpha', values=[1e-1, 1e-2])

    model = keras.Sequential([
                              get_feed_forward_encoder(ff_units, dropout,final_size, activation), 
                              get_classifier_func(hp)   
    ])
    model = get_model_built(model, hp_learning_rate, hp_alpha)
    return model
  return get_tunable_ff

# record stuff

In [75]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Takes an array
def _floatList_feature(value):
  return tf.train.Feature(float_list = tf.train.FloatList(value = value.flatten()))

In [76]:


mode_to_index = {
    0: 0, 
    1: 1, 
    3: 2, 
    4: 3, 
    6: 4
}

def diff(arr):
    res = np.diff(arr)
    return np.concatenate((arr[0:1], res))

def array_lag(arr, n):
    if n < 0:
        return np.pad(arr, [0, -n])[-n:]
    return np.pad(arr, [n, 0])[:-n]

def add_features(df):
    df_copy = df
    
    #df_copy['volume'] = df_copy['flow'].apply(volume_calc)

    df_copy['flow_diff'] = df_copy['flow'].apply(diff)
    df_copy['pressure_diff'] = df_copy['pressure'].apply(diff)

    df_copy['flow_1lag'] = df_copy['flow'].apply(lambda x: array_lag(x, 1))
    df_copy['flow_2lag'] = df_copy['flow'].apply(lambda x: array_lag(x, 2))
    df_copy['flow_3lag'] = df_copy['flow'].apply(lambda x: array_lag(x, 3))
    
    df_copy['pressure_1lag'] = df_copy['pressure'].apply(lambda x: array_lag(x, 1))
    df_copy['pressure_2lag'] = df_copy['pressure'].apply(lambda x: array_lag(x, 2))
    df_copy['pressure_3lag'] = df_copy['pressure'].apply(lambda x: array_lag(x, 3))
    
    df_copy['flow_1oracle'] = df_copy['flow'].apply(lambda x: array_lag(x, -1))
    df_copy['flow_2oracle'] = df_copy['flow'].apply(lambda x: array_lag(x, -2))
    df_copy['flow_3oracle'] = df_copy['flow'].apply(lambda x: array_lag(x, -3))
    
    df_copy['pressure_1oracle'] = df_copy['pressure'].apply(lambda x: array_lag(x, -1))
    df_copy['pressure_2oracle'] = df_copy['pressure'].apply(lambda x: array_lag(x, -2))
    df_copy['pressure_3oracle'] = df_copy['pressure'].apply(lambda x: array_lag(x, -3))

    df_copy['flow_cumsum'] = df_copy['flow'].apply(lambda x: np.cumsum(x))
    df_copy['pressure_cumsum'] = df_copy['pressure'].apply(lambda x: np.cumsum(x))
    
    return df_copy

def writeAsRecord(recordName, file_link):
  data_csv = pd.read_csv(file_link)

  data_np = data_csv[["flow", "pressure", "y"]]
  data_np["flow"] = data_np["flow"].apply(literal_eval) # The csv reader from pandas reads them as strings
  data_np["pressure"] = data_np["pressure"].apply(literal_eval)
  data_np = convert_to_fixed_length(data_np, 202)
  #data_np = add_features(data_np)



  # filtering classes
  data_np = data_np[data_np['y'].isin(mode_to_index.keys())]
  data_np['y'] = data_np['y'].apply(lambda x: mode_to_index[x])

  #data_np = data_np.explode(["flow", "pressure"])
  data_np = data_np.to_numpy()

  # Writing rfRecord
  with tf.io.TFRecordWriter(recordName + '.tfrecord') as tfrecord:
    for idx in range(data_np.shape[0]):
      feature_set = {
      'label'              : _int64_feature(idx),      #tf.train.Feature(int64_list=tf.train.Int64List(value=label)),
      'flow'               : _floatList_feature(np.array(data_np[idx][0])),
      'pressure'           : _floatList_feature(np.array(data_np[idx][1])),
      'y'                  : _float_feature(data_np[idx][2]), #tf.train.Feature(int64_list= tf.train.FloatList(value = feature))
      #'volume'             : _floatList_feature(np.array(data_np[idx][3])),
      #'flow_diff'          : _floatList_feature(np.array(data_np[idx][3])),
      #'pressure_diff'      : _floatList_feature(np.array(data_np[idx][4])),
      #'flow_1lag'          : _floatList_feature(np.array(data_np[idx][5])),
      #'flow_2lag'          : _floatList_feature(np.array(data_np[idx][6])),
      #'flow_3lag'          : _floatList_feature(np.array(data_np[idx][7])),
      #'pressure_1lag'      : _floatList_feature(np.array(data_np[idx][8])),
      #'pressure_2lag'      : _floatList_feature(np.array(data_np[idx][9])),
      #'pressure_3lag'      : _floatList_feature(np.array(data_np[idx][10])),
      #'flow_1oracle'       : _floatList_feature(np.array(data_np[idx][11])),
      #'flow_2oracle'       : _floatList_feature(np.array(data_np[idx][12])),
      #'flow_3oracle'       : _floatList_feature(np.array(data_np[idx][13])),
      #'pressure_1oracle'   : _floatList_feature(np.array(data_np[idx][14])),
      #'pressure_2oracle'   : _floatList_feature(np.array(data_np[idx][15])),
      #'pressure_3oracle'   : _floatList_feature(np.array(data_np[idx][16])),
      #'flow_cumsum'        : _floatList_feature(np.array(data_np[idx][17])),
      #'pressure_cumsum'   : _floatList_feature(np.array(data_np[idx][18])),

      }

      example = tf.train.Example(features = tf.train.Features(feature=feature_set))
      tfrecord.write(example.SerializeToString())

# Writing tfRecords
train_file_link = f"https://drive.google.com/u/0/uc?id=1pHC5l8ww5CnlWpI3RMD7XJAm0lhTMusj&export=download&confirm=t"
test_file_link = f"https://drive.google.com/u/1/uc?id=1hTn-st0A4ZsOPLpHgVilKOgk3IhPnmai&export=download&confirm=t"

writeAsRecord("train_raw", train_file_link)
writeAsRecord("test_raw", test_file_link)

In [77]:
# defining features of record
@tf.function 
def map_fn(serialized_example):
    feature = {
        'label'                 : tf.io.FixedLenFeature(1, tf.int64),
        'flow'                  : tf.io.FixedLenFeature(202, tf.float32),
        'pressure'              : tf.io.FixedLenFeature(202, tf.float32),
        'y'                     : tf.io.FixedLenFeature(1, tf.float32),
        #'volume'                : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_diff'             : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_diff'         : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_1lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_2lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_3lag'             : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_1lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_2lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_3lag'         : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_1oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_2oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_3oracle'          : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_1oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_2oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_3oracle'      : tf.io.FixedLenFeature(200, tf.float32),
        #'flow_cumsum'           : tf.io.FixedLenFeature(200, tf.float32),
        #'pressure_cumsum'       : tf.io.FixedLenFeature(200, tf.float32),        
    }
    return tf.io.parse_single_example(serialized_example, feature)

@tf.function 
def map_all(serialized):
    ex = map_fn(serialized)
    return (tf.stack([ex["flow"], ex["pressure"]],1), ex["y"]) 
                        #ex['volume'], 
                        #ex['flow_diff'], ex['pressure_diff'], 
                        #ex['flow_1lag'], ex['flow_2lag'], ex['flow_3lag'], 
                        #ex['pressure_1lag'], ex['pressure_2lag'], ex['pressure_3lag'], 
                        #ex['flow_1oracle'], ex['flow_2oracle'], ex['flow_3oracle'], 
                        #ex['pressure_1oracle'], ex['pressure_2oracle'], ex['pressure_3oracle'], ex['flow_cumsum'], ex['pressure_cumsum']], 1), ex["y"])
  

@tf.function
def get_dist(targets, n_classes, window_size):
  shape = (n_classes,)
  res = tf.zeros(shape)

  for i in range(window_size):
    idx = tf.gather(targets, i, axis=0)
    mat = tf.sparse.SparseTensor([[idx]], [1.], shape)
    res = tf.sparse.add(res, mat)
    
  res = tf.math.divide(res, window_size)
  return tf.ensure_shape(res, shape)

@tf.function
def map_window(ds, window_size, input_shape):
  feat = tf.zeros([0] + list(input_shape))
  y = tf.zeros([0, ])
  
  for sample in ds:
    ex = map_all(sample)

    features = ex[0]
    features = tf.convert_to_tensor(features)
    features = tf.expand_dims(features, axis=0)

    feat = tf.concat([feat, features], axis=0)
    y = tf.concat([y, ex[1]], axis=0)
    
  feat = tf.ensure_shape(feat, (window_size, 200, 17))
  y = tf.ensure_shape(y, (window_size,))
  return (feat, y)




In [78]:
n_classes = 5
calc_dist = lambda x, y: (x, get_dist(y, n_classes, window_size))

train_set = tf.data.TFRecordDataset('train_raw.tfrecord').shard(2, 0)
val_set = tf.data.TFRecordDataset('train_raw.tfrecord').shard(2, 1)
train_set = train_set.map(map_all)
val_set = val_set.map(map_all)

test_dataset = tf.data.TFRecordDataset('test_raw.tfrecord')
test_dataset = test_dataset.map(map_all)


train_size = sum(1 for _ in train_set)

In [79]:
# transpose 

transpose_set = train_set.map(lambda x,y : (tf.transpose(x),y))
transpose_set = transpose_set.shuffle(10000)
transpose_set = transpose_set.repeat()

transpose_val_set = val_set.map(lambda x,y : (tf.transpose(x),y))


transpose_test_set = test_dataset.map(lambda x,y : (tf.transpose(x),y))

train_set = train_set.shuffle(10000)
train_set = train_set.repeat()

# Tuning

In [80]:
def tune_model(get_model_func, epochs, batch_size, directory, name):
  tuner = kt.Hyperband(
      get_model_func,
      objective='val_loss',
      directory=directory,
      project_name= name,
      hyperband_iterations=1,
      #max_epochs=50
  )

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  tuner.search(transpose_set.batch(batch_size), epochs = epochs, steps_per_epoch = train_size // batch_size, validation_data = transpose_val_set.batch(batch_size), callbacks = [stop_early])
  best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

  print(best_hps.values)
  
  return tuner.hypermodel.build(best_hps)

In [81]:
def tune_model_not_transposed(get_model_func, epochs, batch_size, directory, name):
  tuner = kt.Hyperband(
      get_model_func,
      objective='val_loss',
      directory=directory,
      project_name= name
  )

  stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
  tuner.search(train_set.batch(batch_size), epochs = epochs, steps_per_epoch = train_size // batch_size, validation_data = val_set.batch(batch_size), callbacks = [stop_early])
  best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

  print(best_hps.values)
  
  return tuner.hypermodel.build(best_hps)

# tuning and fitting for report data

## Tuning lstm

In [ ]:
# LSTM

epochs = 50
batch = 1048

tuned_lstm = tune_model(
    get_tunable_lstm_builder(get_tunable_classifier),
    epochs,
    batch, "lstm_cross2", "lstm_cross2"
    )



In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

tuned_lstm.fit(
    transpose_set.batch(batch),
    epochs = epochs,
    steps_per_epoch = train_size // batch,
    validation_data = transpose_val_set.batch(batch),
    callbacks = [stop_early]
    )

model_json = tuned_lstm.to_json()

# Writing json file
with open("lstm_model_cross2.json","w") as json_file:
  json_file.write(model_json)

tuned_lstm.save_weights("lstm_model_cross2.h5")

print("model saved")

tuned_lstm.evaluate(transpose_test_set.batch(batch))

## Tuning CONV

In [ ]:
# CONV

epochs = 50
batch = 1048

tuned_conv = tune_model(
    get_tunable_conv_builder_mod(get_tunable_classifier),
    epochs, 
    batch, "conv_cross2", "conv_cross2"
    )



In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tuned_conv.fit(
      transpose_set.batch(batch),
      epochs = epochs,
      steps_per_epoch = train_size // batch,
      validation_data = transpose_val_set.batch(batch),
      callbacks = [stop_early]
    )

model_json = tuned_conv.to_json()

# Writing json file
with open("conv_model_cross2.json","w") as json_file:
  json_file.write(model_json)

tuned_conv.save_weights("conv_model_cross2.h5")

print("model saved")

tuned_conv.evaluate(transpose_test_set.batch(batch))

## Tuning FF

In [ ]:
# FEED FORWARD

epochs = 50
batch = 1048

tuned_ff = tune_model(
    get_tunable_ff_builder(get_tunable_classifier),
    epochs, 
    batch, "ff_cross2", "ff_cross2"
    )

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
tuned_ff.fit(
      transpose_set.batch(batch),
      epochs = epochs,
      steps_per_epoch = train_size // batch,
      validation_data = transpose_val_set.batch(batch),
      callbacks = [stop_early]
    )

model_json = tuned_ff.to_json()

# Writing json file
with open("ff_model_cross2.json","w") as json_file:
  json_file.write(model_json)

tuned_ff.save_weights("ff_model_cross2.h5")
print("model saved")

tuned_ff.evaluate(transpose_test_set.batch(batch))

# Transformer

In [ ]:
# transformer not made to run with transposed data

epochs = 50
batch = 128

transformer = tune_model_not_transposed(
    get_tunable_transformer,
    epochs, 
    batch, "transformer_cross2", "transformer_cross2"
    )



In [ ]:
# test
tuner = kt.Hyperband(
    get_tunable_transformer,
    objective='val_loss',
    directory="transformer_cross2",
    project_name= "transformer_cross2"
)

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.values)
  
transformer = tuner.hypermodel.build(best_hps)

In [ ]:
epochs = 100 #test
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
transformer.fit(
      train_set.batch(batch),
      epochs = epochs,
      steps_per_epoch = train_size // batch,
      validation_data = val_set.batch(batch),
      callbacks = [stop_early]
    )

model_json = transformer.to_json()

# Writing json file
with open("transformer_cross2.json","w") as json_file:
  json_file.write(model_json)

transformer.save_weights("transformer_cross2.h5")
print("model saved")

transformer.evaluate(test_dataset.batch(batch))

# Loading Model

In [93]:
from sklearn import metrics

def f1_evaluation(file_no_ext, test_set):
    json_file = open(file_no_ext + ".json", 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = keras.models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(file_no_ext + ".h5")

    y_pred = [np.argmax(y) for y in loaded_model.predict(test_set.batch(128))]
    y_true = list(test_set.map(lambda _, y: y[0]))
    
    print(file_no_ext)
    print(metrics.classification_report(y_true,y_pred))
    print(metrics.confusion_matrix(y_true, y_pred))


In [ ]:
f1_evaluation("lstm_model_cross2", transpose_test_set)
f1_evaluation("conv_model_cross2", transpose_test_set)
f1_evaluation("ff_model_cross2", transpose_test_set)
f1_evaluation("transformer_cross2", test_dataset)